Eseguire lo script per scaricare il dataset
Prima di lanciare lo script, registrarsi sul portale kaggle.com e generare la API KEY (file kaggle.json)


In [ ]:
!pip install -q kaggle

from google.colab import files
files.upload()  # carica kaggle.json

!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

!kaggle datasets download -d orvile/pmram-bangladeshi-brain-cancer-mri-dataset
!unzip -q pmram-bangladeshi-brain-cancer-mri-dataset.zip -d brain_mri


Saving kaggle.json to kaggle.json
Dataset URL: https://www.kaggle.com/datasets/orvile/pmram-bangladeshi-brain-cancer-mri-dataset
License(s): CC-BY-SA-4.0


### Utilizzare la struttura delle directory proposta nelle quale riversare il dataset (eventualmente già suddiviso in forma binaria). In alternativa è lo script che effettua la suddivisione e taggatura in "tumor" e "normal".

In [ ]:
import os
import shutil

# Percorso del dataset originale (multiclasse)
original_root = "brain_mri"
# Nuovo percorso per classificazione binaria
#binary_root = "brain_mri_binary"
binary_root = "brain_mri_binary"

tumor_dir = os.path.join(binary_root, "tumor")
normal_dir = os.path.join(binary_root, "normal")

os.makedirs(tumor_dir, exist_ok=True)
os.makedirs(normal_dir, exist_ok=True)

# Tutte le sottocartelle originali
for class_name in os.listdir(original_root):
    class_path = os.path.join(original_root, class_name)
    # Skip if not a directory
    if not os.path.isdir(class_path):
        continue

    # Check if the class_name is the problematic directory
    if class_name == "PMRAM Bangladeshi Brain Cancer - MRI Dataset":
        # Get the actual image directories inside the problematic directory
        for actual_class_name in os.listdir(class_path):
            actual_class_path = os.path.join(class_path, actual_class_name)
            if not os.path.isdir(actual_class_path):
                continue

            # Determine destination based on actual_class_name
            dest_dir = normal_dir if actual_class_name.lower() == "normal" else tumor_dir

            for img_file in os.listdir(actual_class_path):
                src = os.path.join(actual_class_path, img_file)
                # Check if it's a file before copying
                if os.path.isfile(src):
                    dst = os.path.join(dest_dir, f"{actual_class_name}_{img_file}")
                    shutil.copy2(src, dst)

    else:
        # Se la cartella è "normal", copio in /normal, altrimenti in /tumor
        dest_dir = normal_dir if class_name.lower() == "normal" else tumor_dir

        for img_file in os.listdir(class_path):
            src = os.path.join(class_path, img_file)
            #Check if it's a file before copying
            if os.path.isfile(src):
                dst = os.path.join(dest_dir, f"{class_name}_{img_file}")
                shutil.copy2(src, dst)

print("✅ Dataset binario riorganizzato in:", binary_root)

✅ Dataset binario riorganizzato in: brain_mri_binary


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# Esegire nel caso di addestramento BINARIO

import os
from torchvision.datasets import ImageFolder
from torchvision import transforms
from torch.utils.data import DataLoader, random_split

# Trasformazioni applicate a ogni immagine
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
])

# Percorso del dataset riorganizzato per classificazione binaria
#dataset_path = "brain_mri_binary"
dataset_path = "brain_mri_binary"

# Specify supported extensions explicitly
dataset = ImageFolder(dataset_path, transform=transform, is_valid_file=lambda x: x.endswith(('.jpg', '.jpeg', '.png', '.ppm', '.bmp', '.pgm', '.tif', '.tiff', '.webp', '.JPG')))

# Verifica del mapping classi → indici
print("Classi trovate:", dataset.classes)
print("Mapping classi:", dataset.class_to_idx)  # Es: {'normal': 0, 'tumor': 1}

# Suddivisione in train/validation
train_size = int(0.8 * len(dataset))
val_size = len(dataset) - train_size

train_ds, val_ds = random_split(dataset, [train_size, val_size])
train_loader = DataLoader(train_ds, batch_size=32, shuffle=True)
val_loader = DataLoader(val_ds, batch_size=32)

print(f"🧠 Dataset pronto: {len(train_ds)} training - {len(val_ds)} validation")

Classi trovate: ['normal', 'tumor']
Mapping classi: {'normal': 0, 'tumor': 1}
🧠 Dataset pronto: 1090 training - 273 validation


In [ ]:
import os
from torchvision.datasets import ImageFolder
from torchvision import transforms
from torch.utils.data import DataLoader, random_split

transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
])

#dataset = ImageFolder("brain_mri", transform=transform)
dataset = ImageFolder("brain_mri_binary", transform=transform)

train_size = int(0.8 * len(dataset))
val_size = len(dataset) - train_size

train_ds, val_ds = random_split(dataset, [train_size, val_size])
train_loader = DataLoader(train_ds, batch_size=32, shuffle=True)
val_loader = DataLoader(val_ds, batch_size=32)


In [ ]:
import torch
import torchvision.models as models
import torch.nn as nn

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

model = models.resnet18(pretrained=True)
for param in model.parameters():
    param.requires_grad = False

model.fc = nn.Linear(model.fc.in_features, 2)
model = model.to(device)


/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/resnet18-f37072fd.pth" to /root/.cache/torch/hub/checkpoints/resnet18-f37072fd.pth
100%|██████████| 44.7M/44.7M [00:00<00:00, 178MB/s]


In [ ]:
import torch.optim as optim

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.fc.parameters(), lr=0.001)

for epoch in range(5):
    model.train()
    running_loss = 0
    for inputs, labels in train_loader:
        inputs, labels = inputs.to(device), labels.to(device)
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
    print(f"Epoch {epoch+1} - Loss: {running_loss/len(train_loader):.4f}")


Epoch 1 - Loss: 0.4364
Epoch 2 - Loss: 0.2502
Epoch 3 - Loss: 0.1852
Epoch 4 - Loss: 0.1505
Epoch 5 - Loss: 0.1339


In [ ]:
model.eval()
correct = 0
total = 0
with torch.no_grad():
    for inputs, labels in val_loader:
        inputs, labels = inputs.to(device), labels.to(device)
        outputs = model(inputs)
        _, preds = torch.max(outputs, 1)
        correct += (preds == labels).sum().item()
        total += labels.size(0)z

print(f"Accuracy: {correct/total:.2%}")


NameError: name 'model' is not defined

In [ ]:
torch.save(model.state_dict(), "brain_mri_resnet18.pth")
print("Modello salvato in brain_mri_resnet18.pth")



Modello salvato in brain_mri_resnet18.pth


In [ ]:
!pip install onnx
model.load_state_dict(torch.load("brain_mri_resnet18.pth"))
dummy_input = torch.randn(1, 3, 224, 224).to(device)
onnx_path = "brain_mri_resnet18.onnx"

torch.onnx.export(
    model,
    dummy_input,
    onnx_path,
    input_names=["input"],
    output_names=["output"],
    opset_version=11
)

print(f"Modello esportato in formato ONNX: {onnx_path}")


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.0/16.0 MB 109.0 MB/s eta 0:00:00
Modello esportato in formato ONNX: brain_mri_resnet18.onnx


In [ ]:
from google.colab import files
files.download("brain_mri_resnet18.pth")
files.download("brain_mri_resnet18.onnx")


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
# ✅ Installa TensorRT Python bindings (se non già presenti su Colab)
!pip install -q nvidia-pyindex
!pip install -q nvidia-tensorrt --extra-index-url https://pypi.nvidia.com

# ✅ Upload del file ONNX
from google.colab import files
import os

print("📁 Carica un file .onnx dal tuo PC")
uploaded = files.upload()
onnx_file = list(uploaded.keys())[0]
engine_file = onnx_file.replace(".onnx", ".engine")

# ✅ Script di conversione ONNX → TensorRT ENGINE
import tensorrt as trt

TRT_LOGGER = trt.Logger(trt.Logger.INFO)
builder = trt.Builder(TRT_LOGGER)
network = builder.create_network(1 << int(trt.NetworkDefinitionCreationFlag.EXPLICIT_BATCH))
parser = trt.OnnxParser(network, TRT_LOGGER)

print(f"🔍 Parsing ONNX model: {onnx_file}")
with open(onnx_file, 'rb') as model:
    if not parser.parse(model.read()):
        print("❌ Parsing fallito:")
        for i in range(parser.num_errors):
            print(parser.get_error(i))
        raise SystemExit

config = builder.create_builder_config()
config.max_workspace_size = 1 << 28  # 256 MiB

# ⚡ FP16 se disponibile
if builder.platform_has_fast_fp16:
    config.set_flag(trt.BuilderFlag.FP16)
    print("⚙️ FP16 abilitato.")
else:
    print("⚠️ FP16 non supportato, uso FP32.")

print("⚙️ Building engine...")
engine = builder.build_engine(network, config)

# 💾 Salva il motore
with open(engine_file, "wb") as f:
    f.write(engine.serialize())

print(f"✅ Motore TensorRT salvato in: {engine_file}")


📁 Carica un file .onnx dal tuo PC


In [ ]:
# 🔽 Download locale del file engine
from google.colab import files
files.download(engine_file)


NameError: name 'engine_file' is not defined